In [ ]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import io
import tarfile
import os

dates_index_path = "ionosphere-data/jpld/webdataset/dates_index_2010-05-13T00:00:00_2024-07-31T23:45:00"
tar_index_path = "ionosphere-data/jpld/webdataset/tar_files_index"

with open(dates_index_path, "rb") as f:
    dates_index = pickle.load(f)
with open(tar_index_path, "rb") as f:
    tar_files_index = pickle.load(f)

def datetime_to_key(dt):
    return dt.strftime("%Y/%m/%d/%H%M.tecmap.npy")

def patch_tar_path(path):
    # Use the local data directory, not the absolute original
    return os.path.join("ionosphere-data/jpld/webdataset", os.path.basename(path))

def plot_vtec_for_datetime(requested_datetime_str):
    import pandas as pd
    requested_dt = pd.to_datetime(requested_datetime_str)
    key = datetime_to_key(requested_dt)
    if key not in tar_files_index:
        print(f"Exact datetime not found for {key}. Searching nearest...")
        keys = list(tar_files_index.keys())
        key_times = [pd.to_datetime(k[:16], format="%Y/%m/%d/%H%M") for k in keys]
        idx = np.argmin(np.abs([(t - requested_dt).total_seconds() for t in key_times]))
        key = keys[idx]
        requested_dt = key_times[idx]
        print(f"Using nearest available: {key}")
    tar_path, tarinfo = tar_files_index[key]
    local_tar_path = patch_tar_path(tar_path)
    with tarfile.open(local_tar_path, "r") as tar:
        member = tarinfo
        file_obj = tar.extractfile(member)
        array_bytes = file_obj.read()
    vtec_arr = np.load(io.BytesIO(array_bytes))
    plt.figure(figsize=(8, 4))
    plt.imshow(vtec_arr, cmap='viridis', origin='lower', aspect='auto')
    plt.xlabel("Longitude Index")
    plt.ylabel("Latitude Index")
    plt.title(f"VTEC @ {requested_dt}")
    plt.colorbar(label="VTEC (TECU)")
    plt.tight_layout()
    plt.show()

# Example usage:
plot_vtec_for_datetime("2017-03-22T04:45:00")
